A jupyter notebook to test the code generation module

In [9]:
import os
import re
import string
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [10]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(f"The number of calendar data is {len(calendar_data)}.")

with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    # print(json_data[question_index])

print(f"The number of the question is {len(json_data)}.")
print(f"Selected question is: {json_data[0]['question']}.")
print(f"Selected answer is: {json_data[0]['answer']}.")

The number of calendar data is 20.
The number of the question is 46.
Selected question is: How many meetings do I have attended in total?.
Selected answer is: 18.


In [11]:

MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

question_index = 9 # for question test

In [12]:
def get_completion(MODEL, PROMPT):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": PROMPT,
            }
        ],
        model=MODEL, 
    )

    # verify the output
    return chat_completion.choices[0].message.content

In [13]:
def get_prompt(question):
    
    PROMPT = f"""You are provided a calendar. This calendar is a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    This DataFrame calendar_data includes all of your meeting schedule.
    Your task is generate python function to query this dataframe and answer the question. Output a python code by enclosing it in triple backticks. 

    The input have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative; cancelled means that meeting is cancelled.
    - summary: meeting or event topic;
    - start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
    - end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
    - duration: meeting duration (second);
    - attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

    Output is executable Python code by enclosing it in triple backticks:
    ```python
    <your code here>
    ```

    The input of python code is a Pandas dataframe named calendar_data, and the answer is saved in variable answer.

    For example, the output have the following format:
    ```
    import pandas as pd 
    def query(calendar_data):
        return calendar_data[0]
    answer = query(calendar_data)
    ```
    Today's date is '2024-04-02 09:02:30', date format: %Y-%m-%d %H:%M:%S. Today is Tuesday.

    Question to be resolved: {question} 
    """
    return PROMPT



In [ ]:
def get_prompt_for_refine(question, generated_code):
    
    PROMPT = f"""
    
    """
    return PROMPT



In [ ]:
def get_prompt_with_error(question, generated_code, error_info):
    
    PROMPT = f"""
    """
    return PROMPT



In [14]:
def get_python_code(llm_reply_with_code):
    _python_code_re_pattern = "```python\n(.*?)```"
    llm_reply_without_code = re.sub(
                _python_code_re_pattern, "", llm_reply_with_code, flags=re.DOTALL
                )
    python_code_list = re.findall(_python_code_re_pattern, llm_reply_with_code, re.DOTALL)
    return python_code_list[0]

In [15]:
for question in json_data: 
    print(question['question'])
    print(question['answer'])

How many meetings do I have attended in total?
18
How many events do I have scheduled for today?
1
How many attendees are there for the meeting with ID masbk72a24cb0a8k9c7jo0e9s6?
2
What is the longest meeting ID on my calendar?
malrq85j74yb0m3n8j8ro2v5d9
How many events with duration longer than 1 hours?
6
How many meeting were cancelled in total?
2
How many events are scheduled today?
2
How many events are scheduled tomorrow?
0
Do I have me with natalia tomorrow, True or False?
False
How many meeting did I have yestaerday ?
0
How many meetings do I have for next 3 days?
2
What is the topic of my last meeting?
meeting topic: weekly progress review
What is the topic of my next meeting?
Database Architecture 
What is the shortest meeting duration?
1200
How many confirmed meetings are scheduled for tomorrow?
0
Who is the most frequent attendee in meetings?
walter@ninjatech.ai(accepted)
What is the average meeting duration?
3660
What is my latest scheduled meeting for today? tell me meeti

In [16]:
result = {'success': 0, 'wrong_answer': 0, 'error': 0}
question_answer_summary = {'question': '',
                           'generated_code': '',
                           'true_answer':'', 
                           'answer': ''
                        }
result_list = []

for q_a_pair in json_data: 
    PROMPT = get_prompt(q_a_pair['question'])
    llm_reply_with_code = get_completion(MODEL, PROMPT)
    python_code_list = get_python_code(llm_reply_with_code)
    
    question_answer_summary['question'] = q_a_pair['question']
    question_answer_summary['true_answer'] = q_a_pair['answer']
    question_answer_summary['generated_code'] = python_code_list
    
    print(f"""Question: {q_a_pair['question']}\n True answer: {q_a_pair['answer']}""")

    try:
        exec(python_code_list)
    except Exception as E:
        question_answer_summary['answer'] = repr(E)
        print(f"""Non-executable code: {E}""")
        result['error'] += 1
        pass
    else:
        # print(f"""generated answer: {answer}""")
        if q_a_pair['answer'] == answer:
            question_answer_summary['answer'] = 'success'
            result['success'] += 1
        else:   
            question_answer_summary['answer'] = repr(answer)
            print(f"""Wrong answer: {answer}""")
            result['wrong_answer'] += 1
    
    result_list.append(question_answer_summary.copy())
    print(result)
    
# print(json.dumps(result_list, indent=4))
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(result_list, f, indent=4)
    

Question: How many meetings do I have attended in total?
 True answer: 18
{'success': 0, 'wrong_answer': 1, 'error': 0}
Question: How many events do I have scheduled for today?
 True answer: 1
{'success': 0, 'wrong_answer': 1, 'error': 1}
Question: How many attendees are there for the meeting with ID masbk72a24cb0a8k9c7jo0e9s6?
 True answer: 2
{'success': 0, 'wrong_answer': 2, 'error': 1}
Question: What is the longest meeting ID on my calendar?
 True answer: malrq85j74yb0m3n8j8ro2v5d9
{'success': 1, 'wrong_answer': 2, 'error': 1}
Question: How many events with duration longer than 1 hours?
 True answer: 6
{'success': 2, 'wrong_answer': 2, 'error': 1}
Question: How many meeting were cancelled in total?
 True answer: 2
{'success': 3, 'wrong_answer': 2, 'error': 1}
Question: How many events are scheduled today?
 True answer: 2
{'success': 4, 'wrong_answer': 2, 'error': 1}
Question: How many events are scheduled tomorrow?
 True answer: 0
{'success': 5, 'wrong_answer': 2, 'error': 1}
Questi

<string>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Question: Find the duration of meeting with id 'mahsm54f88qb0j5o8p3uo0a9v8'.?
 True answer: 3600


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
print(result_list) 

[{'question': 'How many meetings do I have attended in total?', 'generated_code': "import pandas as pd \n\ndef total_meetings(calendar_data):\n    # based on the logic that cancelled meetings won't be attended, we filtered out cancelled meetings\n    not_cancelled_meetings = calendar_data[calendar_data['status'] != 'cancelled']  \n    total_meetings_attended = not_cancelled_meetings.shape[0]\n    return total_meetings_attended\n\nanswer = total_meetings(calendar_data)\n", 'true_answer': 18, 'answer': 'success'}, {'question': 'How many events do I have scheduled for today?', 'generated_code': "import pandas as pd\nfrom datetime import datetime\n\ndef num_of_events_today(calendar_data):\n    # Convert the 'start' column to datetime datatype\n    calendar_data['start'] = pd.to_datetime(calendar_data['start'])\n\n    # Filter the dataframe to include the events of today\n    events_today = calendar_data[(calendar_data['start'].dt.date == datetime.now().date())]\n\n    return events_today.s